In [1]:
import numpy as np
import cv2 as cv
import os
import tqdm
import matplotlib.pyplot as plt
from PIL import Image

# Data Loading

In [3]:
COMMON_PART = '/home/bohdan/histologie/data/doctor_annotations/'
PATH_TO_COLORFUL_MASKS = os.path.join(COMMON_PART, 'preprocessed/vis/visible/')
PATH_TO_ORIGINAL_IMAGES = os.path.join(COMMON_PART, 'x')
PATH_TO_BINARY_MASKS = os.path.join(COMMON_PART, 'visible_binary_masks')
PATH_TO_COLORFUL_CONTOURS = os.path.join(COMMON_PART, 'colored_contours')
PATH_TO_EDITED_MASKS = os.path.join(COMMON_PART, 'preprocessed/y')
PATH_TO_EDITED_IMGS = os.path.join(COMMON_PART, 'preprocessed/x')

In [4]:
colorful_mask_names = os.listdir(PATH_TO_COLORFUL_MASKS)
original_img_names = os.listdir(PATH_TO_ORIGINAL_IMAGES)
binary_mask_names = os.listdir(PATH_TO_BINARY_MASKS)
colored_contours_names = os.listdir(PATH_TO_COLORFUL_CONTOURS)
edited_masks_names = os.listdir(PATH_TO_EDITED_MASKS)
edited_imgs_names = os.listdir(PATH_TO_EDITED_IMGS)

In [5]:
# original_images = [cv.imread(os.path.join(PATH_TO_ORIGINAL_IMAGES, original_img_name))[..., ::-1]
#                    for original_img_name in colorful_mask_names]
colorful_masks = [cv.imread(os.path.join(PATH_TO_COLORFUL_MASKS, colorful_mask_name))[..., ::-1]
                  for colorful_mask_name in colorful_mask_names]
# binary_masks = [cv.imread(os.path.join(PATH_TO_BINARY_MASKS, binary_mask_name))[..., ::-1] 
#                 for binary_mask_name in colorful_mask_names]
# colored_contours = [cv.imread(os.path.join(PATH_TO_COLORFUL_CONTOURS, colored_contour_name))[..., ::-1]
#                    for colored_contour_name in colorful_mask_names]

edited_imgs = [cv.imread(os.path.join(PATH_TO_EDITED_IMGS, edited_img_name))[..., ::-1]
                   for edited_img_name in edited_imgs_names]
edited_masks = [cv.imread(os.path.join(PATH_TO_EDITED_MASKS, edited_mask_name))[..., ::-1]
                   for edited_mask_name in edited_masks_names]


# Original imgs with doctors contours VS Floodfilled

In [ ]:
PATH_TO_SAVE = '/home/bohdan/histologie/data/doctor_annotations/visualistaion/orig_imgs_contours_vs_floodfilled'
if not os.path.exists(PATH_TO_SAVE):
    os.mkdir(PATH_TO_SAVE)

In [ ]:
for i in range(len(colored_contours_names)):
    cur_merged_img = cv.add(colored_contours[i], original_images[i])
    cur_merged_img = np.concatenate((cur_merged_img, colorful_masks[i]), axis=1)
    Image.fromarray(cur_merged_img).save(os.path.join(PATH_TO_SAVE, colorful_mask_names[i]))

# Original imgs with binary mask contours

## Binary mask preprocessing

In [ ]:
kernel = np.ones((10,10), 'uint8')
for i in range(len(binary_mask_names) - 10):
#     morphology = cv.morphologyEx(np.copy(binary_masks[i]), cv.MORPH_OPEN, kernel)
    plt.figure(figsize=(10,10))
#     plt.subplot(1,2,1)
    plt.title(binary_mask_names[i], fontweight='bold', fontsize=30)
    plt.imshow(original_img_names[i])
    plt.subplot(1,2,2)
    plt.imshow(binary_masks[i])

## Visualisation

In [ ]:
PATH_TO_SAVE = '/home/bohdan/histologie/data/doctor_annotations/visualistaion/orig_imgs_with_binary_mask_contours'
if not os.path.exists(PATH_TO_SAVE):
    os.mkdir(PATH_TO_SAVE)

In [ ]:
kernel = np.ones((10,10), 'uint8')
for i in range(len(colorful_mask_names)):
    cur_mask = cv.morphologyEx(np.copy(binary_masks[i]), cv.MORPH_OPEN, kernel)
    cur_mask = binary_masks[i]
    cur_mask = cv.cvtColor(cur_mask, cv.COLOR_RGB2GRAY)
    (_, contours, hierarchy) = cv.findContours(cur_mask, cv.RETR_LIST, cv.CHAIN_APPROX_NONE)
#     plt.figure(figsize=(20,10))
    orig_with_contours = np.copy(original_images[i])
    cv.drawContours(orig_with_contours, contours, -1, (255,255,255), 3)
    result = np.concatenate((orig_with_contours, cv.cvtColor(cur_mask, cv.COLOR_GRAY2BGR)), axis=1)
#     plt.title(original_img_names[i], fontweight='bold', fontsize=30)
#     plt.imshow(result)
    Image.fromarray(result).save(os.path.join(PATH_TO_SAVE, colorful_mask_names[i]))

# Edited Images + contours of edited masks VS edited masks

In [6]:
def draw_colorful_contours(img, cur_mask):
    cur_img = np.copy(img)
    colors = [(255,0,0), (0,255,0), (255,0,127),
          (255,0,255),(128,130,128)]
    for (ind, cl) in enumerate([1,2,6,9]):
        cur_class = cv.cvtColor((cur_mask == cl).astype('uint8'), cv.COLOR_BGR2GRAY)
        (_, contours, _) = cv.findContours(cur_class,cv.RETR_LIST, cv.CHAIN_APPROX_NONE)
        cv.drawContours(cur_img, contours, -1, colors[ind], 10)
    return cur_img

In [8]:
PATH_TO_SAVE = '/home/bohdan/histologie/data/doctor_annotations/visualistaion/preprocessed_masks/'
for i in range(len(edited_imgs)):
    cur_img = np.concatenate((draw_colorful_contours(edited_imgs[i], edited_masks[i]), colorful_masks[i]), axis=1)
    Image.fromarray(cur_img).save(os.path.join(PATH_TO_SAVE, edited_imgs_names[i]))